In [42]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

model = OllamaLLM(model="gemma3:4b")

In [43]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回復客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞)，我會自行手動增加。"),
    ("human","請回覆這封客戶郵件:\n\n{email_content}")
])

In [44]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    time_format_template = "{y}%Y-%m-%d %H:%M"
    current_time = datetime.datetime.now()
    date_str = current_time.strftime(time_format_template).format(
        y='中華民國,西元', # 用來替換 {y}
    )
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:{date_str}
"""

    return formatted_reply

format_email = RunnableLambda(format_email_reply)

In [45]:
chain = (prompt_template
        | model
        | StrOutputParser()
        | format_email
        )
        

In [46]:
customer_email = """您好:
我最近購買了櫃公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎?我已經購買一週了。

期待您的回覆!

王小明"""

In [47]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

王小明您好，

非常抱歉讓您有不愉快的購物經驗。我們深感遺憾聽到您收到的產品有損壞且功能與說明書不符的情況。

請您放心，我們會盡力協助您處理這件事情。請您提供更多關於產品型號、損壞情況的細節，以及說明書上的不符部分，以便我們更快速地釐清問題並提供最適合的解決方案。

我們非常重視您的回饋，並會盡快為您安排處理。


感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:中華民國,西元2025-10-18 15:06

